In [17]:
import torch
import cv2
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def test_image_show(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.imshow(image)
    plt.axis('off')
    plt.show()

In [3]:
# def show_predictions(predictions, image_path, iamge_name):
#     # Load the image
#     image = cv2.imread(image_path)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

#     # Create a figure and axis
#     fig, ax = plt.subplots(1, figsize=(12, 8))
#     ax.imshow(image)

#     # Plot each prediction
#     for pred in predictions:
#         label, score, bbox = pred  # unpack tuple

#         # Draw bounding box
#         rect = plt.Rectangle((bbox[0], bbox[1]), bbox[2] - bbox[0], bbox[3] - bbox[1],
#                              linewidth=2, edgecolor='red', facecolor='none')
#         ax.add_patch(rect)

#         # Add label and score
#         ax.text(bbox[0], bbox[1] - 10, f'{label}: {score:.2f}', color='red', fontsize=12, 
#                 bbox=dict(facecolor='white', alpha=0.5, edgecolor='none'))

#     plt.axis('off')
#     plt.show()
#     fig.savefig(os.path.join('/ExDark-Object-Detection/output', image_name))
#     plt.close(fig)


In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

def detect_image_yolov5(image_path, model_path='/ExDark-Object-Detection/models/YOLOv5.pt'):
    # Load YOLOv5 model
    model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path, device=device)
    model.eval()

    # Đọc ảnh bằng OpenCV
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Không thể đọc ảnh từ đường dẫn: {image_path}")

    # Convert BGR -> RGB vì OpenCV dùng BGR, YOLOv5 dùng RGB
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Dự đoán
    results = model(img_rgb)

    # Kết quả dạng pandas DataFrame
    df = results.pandas().xyxy[0]  # bounding boxes theo dạng (x1, y1, x2, y2)

    output = []
    for _, row in df.iterrows():
        label = row['name']
        conf = float(row['confidence'])
        bbox = (int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax']))
        output.append((label, conf, bbox))
    
    return output


In [26]:
# Use the default YOLOv5s model if custom model is not available
output = {}

# Get all of the images in the input directory
input_dir = '/ExDark-Object-Detection/input'

if not os.path.exists(input_dir):
    raise ValueError(f"Input directory does not exist: {input_dir}")
input_images = [f for f in os.listdir(input_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

if not input_images:
    raise ValueError(f"No images found in input directory: {input_dir}")

for image_name in input_images:
    image_path = os.path.join(input_dir, image_name)
    test_image_show(image_path)  # Show the image
    pred = detect_image_yolov5(image_path)  # Get predictions
    output[image_name] = pred  # Store predictions

Using cache found in C:\Users\DELL Pre 7670/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-6-4 Python-3.12.4 torch-2.6.0+cu124 CUDA:0 (NVIDIA RTX A1000 Laptop GPU, 4096MiB)

Fusing layers... 
custom_YOLOv5s summary: 232 layers, 7257306 parameters, 0 gradients
Adding AutoShape... 
C:\Users\DELL Pre 7670/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
Using cache found in C:\Users\DELL Pre 7670/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-6-4 Python-3.12.4 torch-2.6.0+cu124 CUDA:0 (NVIDIA RTX A1000 Laptop GPU, 4096MiB)

Fusing layers... 
custom_YOLOv5s summary: 232 layers, 7257306 parameters, 0 gradients
Adding AutoShape... 
C:\Users\DELL Pre 7670/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.aut

In [27]:
for image_name, predictions in output.items():
    print(f"Predictions for {image_name}:")
    for label, conf, bbox in predictions:
        print(f"  Label: {label}, Confidence: {conf:.2f}, BBox: {bbox}")
    

Predictions for special_test.jpg:
Predictions for test_1.jpg:
  Label: People, Confidence: 0.90, BBox: (81, 107, 102, 163)
  Label: People, Confidence: 0.86, BBox: (124, 106, 148, 160)
Predictions for test_10.jpg:
  Label: People, Confidence: 0.90, BBox: (481, 87, 601, 266)
Predictions for test_11.jpg:
  Label: Bus, Confidence: 0.94, BBox: (1204, 457, 2263, 1766)
  Label: Car, Confidence: 0.63, BBox: (394, 1213, 678, 1371)
  Label: Car, Confidence: 0.53, BBox: (955, 1141, 1217, 1308)
Predictions for test_13.jpg:
  Label: People, Confidence: 0.67, BBox: (381, 197, 526, 337)
  Label: People, Confidence: 0.66, BBox: (119, 119, 194, 248)
  Label: People, Confidence: 0.44, BBox: (116, 114, 276, 261)
Predictions for test_14.jpg:
Predictions for test_15.jpg:
Predictions for test_16.jpg:
  Label: People, Confidence: 0.69, BBox: (129, 62, 480, 470)
Predictions for test_17.jpg:
  Label: Motorbike, Confidence: 0.60, BBox: (488, 1272, 3246, 5408)
  Label: People, Confidence: 0.27, BBox: (892, 944,

In [28]:
print(output)

{'special_test.jpg': [], 'test_1.jpg': [('People', 0.9019312858581543, (81, 107, 102, 163)), ('People', 0.860504150390625, (124, 106, 148, 160))], 'test_10.jpg': [('People', 0.8970347046852112, (481, 87, 601, 266))], 'test_11.jpg': [('Bus', 0.9393107891082764, (1204, 457, 2263, 1766)), ('Car', 0.6256062388420105, (394, 1213, 678, 1371)), ('Car', 0.5270803570747375, (955, 1141, 1217, 1308))], 'test_13.jpg': [('People', 0.670373260974884, (381, 197, 526, 337)), ('People', 0.6604030132293701, (119, 119, 194, 248)), ('People', 0.4448811709880829, (116, 114, 276, 261))], 'test_14.jpg': [], 'test_15.jpg': [], 'test_16.jpg': [('People', 0.6936938166618347, (129, 62, 480, 470))], 'test_17.jpg': [('Motorbike', 0.6047858595848083, (488, 1272, 3246, 5408)), ('People', 0.26755160093307495, (892, 944, 2839, 5408))], 'test_18.jpg': [('Motorbike', 0.9067729115486145, (347, 182, 489, 336))], 'test_2.jpg': [('Car', 0.9115651249885559, (229, 448, 362, 551)), ('Car', 0.8812650442123413, (490, 447, 593, 5

In [ ]:
output = {'special_test.jpg': [], 'test_1.jpg': [('People', 0.9019312858581543, (81, 107, 102, 163)), ('People', 0.860504150390625, (124, 106, 148, 160))], 'test_10.jpg': [('People', 0.8970347046852112, (481, 87, 601, 266))], 'test_11.jpg': [('Bus', 0.9393107891082764, (1204, 457, 2263, 1766)), ('Car', 0.6256062388420105, (394, 1213, 678, 1371)), ('Car', 0.5270803570747375, (955, 1141, 1217, 1308))], 'test_13.jpg': [('People', 0.670373260974884, (381, 197, 526, 337)), ('People', 0.6604030132293701, (119, 119, 194, 248)), ('People', 0.4448811709880829, (116, 114, 276, 261))], 'test_14.jpg': [], 'test_15.jpg': [], 'test_16.jpg': [('People', 0.6936938166618347, (129, 62, 480, 470))], 'test_17.jpg': [('Motorbike', 0.6047858595848083, (488, 1272, 3246, 5408)), ('People', 0.26755160093307495, (892, 944, 2839, 5408))], 'test_18.jpg': [('Motorbike', 0.9067729115486145, (347, 182, 489, 336))], 'test_2.jpg': [('Car', 0.9115651249885559, (229, 448, 362, 551)), ('Car', 0.8812650442123413, (490, 447, 593, 521)), ('Car', 0.8443943858146667, (611, 436, 692, 509)), ('People', 0.8051103353500366, (402, 448, 614, 970)), ('Car', 0.7979825735092163, (566, 446, 631, 517)), ('Car', 0.7658812403678894, (379, 452, 495, 533)), ('People', 0.36156365275382996, (121, 431, 180, 547)), ('Car', 0.2831708788871765, (687, 449, 730, 485)), ('Car', 0.25104260444641113, (650, 441, 720, 491))], 'test_3.jpg': [('People', 0.615678071975708, (103, 25, 128, 75)), ('Bicycle', 0.5417863130569458, (0, 12, 91, 165)), ('People', 0.3908799886703491, (33, 2, 84, 141))], 'test_4.jpg': [], 'test_5.jpg': [('Car', 0.9175679683685303, (777, 762, 1834, 1270))], 'test_6.jpg': [('People', 0.2697101831436157, (956, 879, 1098, 1193))], 'test_7.jpg': [('Car', 0.5361272096633911, (93, 67, 132, 85))], 'test_8.jpg': [], 'test_9.jpg': [('People', 0.7931485176086426, (114, 106, 129, 125))]}
input_path =  '/ExDark-Object-Detection/input'
output_path = '/ExDark-Object-Detection/output/YOLOv5'

def plot_predictions(output, input_path, output_path):
    for image_name, predictions in output.items():
        # Load the image
        image_path = os.path.join(input_path, image_name)
        if not os.path.exists(image_path):
            print(f"Image {image_name} does not exist in {input_path}. Skipping.")
            continue
        
        image = plt.imread(image_path)
        
        # Create a figure and axis
        fig, ax = plt.subplots(1, figsize=(12, 8))
        ax.imshow(image)

        # Plot each prediction
        for label, score, bbox in predictions:
            x1, y1, x2, y2 = bbox
            width = x2 - x1
            height = y2 - y1
            
            # Draw rectangle around the object
            rect = plt.Rectangle((x1, y1), width, height, linewidth=2, edgecolor='red', facecolor='none')
            ax.add_patch(rect)
            
            # Add label and score
            ax.text(x1, y1 - 10, f"{label} ({score:.2f})", color='white', fontsize=12)

        # Set title and show plot
        ax.set_title(f"Predictions for {image_name}")
        plt.axis('off')
        
        # Save the plot to output directory
        output_image_path = os.path.join(output_path, f"predicted_{image_name}")
        plt.savefig(output_image_path)
        plt.close(fig)
        
        print(f"Saved predictions for {image_name} to {output_image_path}")
        
if __name__ == "__main__":
    # Ensure output directory exists
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    # Plot predictions
    plot_predictions(output, input_path, output_path)